In [1]:
import os
import gc
import time
import pandas as pd
import numpy as np

import geopandas as gpd
import rioxarray as rioxr
import rasterio

import data_sampling_workflow.sample_rasters as sr
from rasterio.crs import CRS

import planetary_computer as pc

from skimage.feature import graycomatrix, graycoprops
import raster_to_features as rf

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

window_r = 1  #in pixels

distances = [1]
angles = [np.pi/2]
props = ['contrast']#, 'correlation']

# ***************************************************
# ***************************************************

In [3]:
itemids = pd.read_csv(sr.path_to_aoi_itemids_csv())
itemids = itemids[ (itemids.aoi_name == 'point_conception') & (itemids.year == 2020)]
itemid = itemids.itemid[0]
raster = rf.rioxr_from_itemid(itemid)

In [66]:
# BENCHAMRK: ITERATING THROUGH EACH PIXEL
y_len = 5
x_len = 5
band_n = 0


t0 = time.time()
contrast2 = np.zeros((y_len,x_len))
for y in range(y_len):
    for x in range(x_len):
        window = raster[:, y-window_r:y+window_r+1, x-window_r:x+window_r+1]      # MAKE THIS A DIRECT COMPARISON BEFORE MAKING THE WINDOW OR START x,y AT RIGHT INDICES
        if (window.shape[1] != 2*window_r+1) | (window.shape[2] != 2*window_r+1):
            contrast2[y,x] = np.nan
        else:
            glcm = graycomatrix(window[band_n], distances=distances, angles=angles)
#            print(np.nonzero(glcm), '\n')
            contrast2[y,x] = graycoprops(glcm, 'contrast')[0,0]
            
print(time.time() - t0)

(array([73, 74, 76, 78, 79, 86]), array([74, 98, 78, 88, 86, 87]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0])) 

(array([76, 78, 79, 86, 88, 89]), array([78, 88, 86, 87, 95, 88]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0])) 

(array([ 79,  86,  88,  89,  94, 112]), array([ 86,  87,  95,  88, 108,  94]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0])) 

(array([ 88,  89,  94, 101, 112, 120]), array([ 95,  88, 108, 112,  94, 101]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0])) 

(array([74, 78, 86, 87, 88, 98]), array([ 98,  88,  87, 104,  99, 112]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0])) 

(array([78, 86, 87, 88, 88, 95]), array([ 88,  87, 104,  95,  99, 109]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0])) 

(array([ 86,  87,  88,  94,  95, 108]), array([ 87, 104,  95, 108, 109, 120]), array([0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0])) 

(array([ 88,  94,  95, 101, 108, 112]), array([ 95, 108, 109, 112, 120, 119]), array([0, 0,

In [101]:
#The value P[i,j,d,theta] is the number of times that gray-level j occurs at a distance d and at an angle theta from gray-level i.

def L_out_R_in(raster, x,y, prev_glcm, window_r):
#    x,y = column and row in raster of center of window over which prev_glcm was calculated NOT
    new = prev_glcm   # this is 4D array of shape 256, 256, 1, 1 (assuming 1 direction-N and 1 angle pi/2)
    top = y - window_r
    bott = y + window_r
        
    # get i,j occurrences in Lcolumn... 
    #Lx = x - window_r
    Lx = x - window_r -1
#    pairs = np.column_stack((raster[top+1:bott+1 ,Lx], raster[top:bott ,Lx]))
    pairs = np.column_stack((raster[top:bott ,Lx],raster[top+1:bott+1 ,Lx]))
    # print('out')
    # print(pairs, '\n')
    u_pairs, counts = np.unique(pairs, return_counts=True, axis=0)
    # ... and substract them 
    for pair, count in zip(u_pairs, counts):
        new[pair[0],pair[1],0,0] -= count
        
    # get i,j occurrences in Rcolumn...     
#    Rx = x + window_r+1
    Rx = x + window_r
    pairs = np.column_stack((raster[top+1:bott+1 ,Rx], raster[top:bott ,Rx]))
    # print('in')
    # print(pairs, '\n')    
    u_pairs, counts = np.unique(pairs, return_counts=True, axis=0)
    # ... and add them 
    for pair, count in zip(u_pairs, counts):
        new[pair[0],pair[1],0,0] += count

    return new
    

In [76]:
# a = np.array([0,1,2,1,2,2])
# raster = np.tile(a,(6,1))
# raster

In [77]:
# x,y = 1,1
# window_r = 1
# window = np.tile(np.array([0,1,2],dtype='uint8'),(3,1))
# prev_glcm = graycomatrix(window, distances=distances, angles=angles, levels=3)
# for x in range(1,4):
#     prev_glcm = L_out_R_in(raster, x,y, prev_glcm, window_r)
#     print(prev_glcm[:,:,0,0], '\n')

In [107]:
y_len = 5
x_len = 5
contrast = np.zeros((y_len,x_len))

t0 = time.time()

for y in range(y_len):    
    for x in range(x_len):
        window = raster[:, y-window_r:y+window_r+1, x-window_r:x+window_r+1]      # MAKE THIS A DIRECT COMPARISON BEFORE MAKING THE WINDOW OR START x,y AT RIGHT INDICES
        
        if (window.shape[1] != 2*window_r+1) | (window.shape[2] != 2*window_r+1):
            contrast[y,x] = np.nan
        else:
 #           print(window[band_n].data, '\n')
            if x == window_r:
                this_glcm = graycomatrix(window[band_n].squeeze(), distances=distances, angles=angles)
            else:
                this_glcm = L_out_R_in(raster[band_n].squeeze(), x, y, prev_glcm, window_r)
#            print(np.nonzero(this_glcm), '\n')                
            contrast[y,x] = graycoprops(this_glcm, 'contrast')[0,0]
            prev_glcm = this_glcm
print(time.time() - t0)

0.08235859870910645


In [103]:
contrast2

array([[         nan,          nan,          nan,          nan,
                 nan],
       [         nan, 121.83333333,  34.        , 103.33333333,
        175.33333333],
       [         nan, 213.83333333, 126.        , 145.83333333,
        125.83333333],
       [         nan, 471.83333333, 512.66666667, 436.5       ,
        279.16666667],
       [         nan, 474.5       , 563.33333333, 454.        ,
        277.16666667]])

In [106]:
contrast

array([[         nan,          nan,          nan,          nan,
                 nan],
       [         nan, 121.83333333,  34.        , 103.33333333,
        175.33333333],
       [         nan, 213.83333333, 126.        , 145.83333333,
        125.83333333],
       [         nan, 471.83333333, 512.66666667, 436.5       ,
        279.16666667],
       [         nan, 474.5       , 563.33333333, 454.        ,
        277.16666667]])